In [4]:
import pandas as pd

# 常量定义
TEAM_KEYWORDS = r'\b(Team|Relay|Doubles|Pairs|Crew|Squad|Group|Combined)\b'

def process_medal_data(input_file, output_file):
    # 数据读取与预处理
    df = pd.read_csv(input_file, keep_default_na=False)
    
    # 标记团体项目
    df['is_team'] = df['Event'].str.contains(
        TEAM_KEYWORDS, 
        case=False, 
        regex=True,
        na=False
    )
    
    # 处理团体项目去重
    team_dedup = df[df['is_team']].drop_duplicates(
        subset=['NOC', 'Year', 'Sport', 'Event'],
        keep='first'
    )
    
    # 合并数据
    final_df = pd.concat([
        df[~df['is_team']],
        team_dedup
    ], ignore_index=True)
    
    # 分组统计（核心改进）
    result = final_df.groupby(
        ['NOC', 'Year', 'Sport'],  # 按国家、年份、运动分组
        as_index=False             # 保留原始列作为字段
    ).agg({
        'Total': 'sum',           # 总奖牌数求和
        'Gold': 'sum',            # 金牌数求和
        'Silver': 'sum',          # 银牌数求和
        'Bronze': 'sum'           # 铜牌数求和
    })
    
    # 输出结果
    result.to_csv(output_file, index=False)
    print(f"处理完成，结果已保存至 {output_file}")

# 执行示例
process_medal_data(r"C:\Users\NANA\.jupyter\JupyterProject\MCM_02\code\data\athletes_1.csv", r"C:\Users\NANA\.jupyter\JupyterProject\MCM_02\olympic_medal_summary.csv")

C:\Users\NANA\AppData\Local\Temp\ipykernel_44392\3935887476.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['is_team'] = df['Event'].str.contains(


处理完成，结果已保存至 C:\Users\NANA\.jupyter\JupyterProject\MCM_02\olympic_medal_summary.csv


In [6]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import STL
import warnings

# 禁用警告
warnings.filterwarnings("ignore")

def process_country_sport(group):
    """处理单个国家-运动组合"""
    try:
        noc = group['NOC'].iloc[0]
        sport = group['Sport'].iloc[0]
        
        # 生成完整奥运时间轴
        min_year = group['Year'].min()
        max_year = group['Year'].max()
        full_years = pd.Index(
            np.arange(min_year, max_year + 4, 4),
            name='Year'
        )
        
        # 创建完整时间序列
        full_df = pd.DataFrame({'Year': full_years})
        merged = pd.merge(full_df, group, on='Year', how='left')
        
        # 标记参赛情况并填充0值
        merged['participated'] = merged['Total'].notna()
        merged.fillna({'Total': 0, 'Gold': 0, 'Silver': 0, 'Bronze': 0}, inplace=True)
        merged[['Total', 'Gold', 'Silver', 'Bronze']] = merged[['Total', 'Gold', 'Silver', 'Bronze']].astype(int)
        
        # STL分解（针对总奖牌数）
        ts = merged.set_index('Year')['Total']
        res = STL(
            ts,
            period=4,       # 奥运周期
            seasonal=5,     # 5届季节性窗口
            trend=7,        # 7届趋势窗口
            robust=True
        ).fit()
        
        # 合并分解结果
        result = merged.copy()
        result['Trend'] = res.trend.values
        result['Seasonal'] = res.seasonal.values
        result['Residual'] = res.resid.values
        
        # 只保留原始参赛年份
        final = result[result['participated']].drop(columns=['participated'])
        final['NOC'] = noc
        final['Sport'] = sport
        
        return final[['NOC', 'Year', 'Sport', 'Total', 'Gold', 'Silver', 'Bronze', 'Trend', 'Seasonal', 'Residual']]
    
    except Exception as e:
        print(f"Error processing {noc}-{sport}: {str(e)}")
        return None

def main(input_file, output_file):
    # 读取数据（优化内存使用）
    df = pd.read_csv(
        input_file,
        dtype={
            'NOC': 'category',
            'Year': int,
            'Sport': 'category',
            'Total': 'int16',
            'Gold': 'int8',
            'Silver': 'int8',
            'Bronze': 'int8'
        }
    )
    
    # 单进程处理
    results = []
    for (noc, sport), group in df.groupby(['NOC', 'Sport']):
        if len(group) >= 3:  # 至少需要3个数据点
            result = process_country_sport(group)
            if result is not None:
                results.append(result)
                print(f"已处理 {noc}-{sport} ({len(group)}条数据)")
    
    # 合并结果
    if results:
        final_df = pd.concat(results, ignore_index=True)
        final_df.to_csv(output_file, index=False)
        print(f"处理完成！共处理{len(final_df)}条有效记录")
    else:
        print("未生成有效结果")

if __name__ == "__main__":
    # 使用示例路径
    input_path = r"C:\Users\NANA\.jupyter\JupyterProject\MCM_02\code\data\olympic_medal_summary.csv"
    output_path = r"C:\Users\NANA\.jupyter\JupyterProject\MCM_02\decomposition_results.csv"
    
    # 执行主程序
    main(input_path, output_path)

已处理 AFG-Athletics (9条数据)
已处理 AFG-Boxing (3条数据)
已处理 AFG-Hockey (3条数据)
已处理 AFG-Judo (4条数据)
已处理 AFG-Taekwondo (3条数据)
已处理 AFG-Wrestling (7条数据)
已处理 AHO-Athletics (7条数据)
已处理 AHO-Sailing (4条数据)
已处理 AHO-Shooting (6条数据)
已处理 AHO-Swimming (5条数据)
已处理 AHO-Weightlifting (4条数据)
已处理 ALB-Athletics (9条数据)
已处理 ALB-Judo (3条数据)
已处理 ALB-Shooting (8条数据)
已处理 ALB-Swimming (7条数据)
已处理 ALB-Weightlifting (9条数据)
已处理 ALB-Wrestling (4条数据)
已处理 ALG-Athletics (13条数据)
已处理 ALG-Boxing (14条数据)
已处理 ALG-Cycling (4条数据)
已处理 ALG-Fencing (9条数据)
已处理 ALG-Gymnastics (3条数据)
已处理 ALG-Handball (4条数据)
已处理 ALG-Judo (11条数据)
已处理 ALG-Rowing (8条数据)
已处理 ALG-Sailing (3条数据)
已处理 ALG-Shooting (4条数据)
已处理 ALG-Swimming (10条数据)
已处理 ALG-Table Tennis (5条数据)
已处理 ALG-Volleyball (3条数据)
已处理 ALG-Weightlifting (12条数据)
已处理 ALG-Wrestling (10条数据)
已处理 AND-Athletics (10条数据)
已处理 AND-Judo (6条数据)
已处理 AND-Shooting (9条数据)
已处理 AND-Swimming (6条数据)
已处理 ANG-Athletics (11条数据)
已处理 ANG-Basketball (6条数据)
已处理 ANG-Boxing (3条数据)
已处理 ANG-Handball (8条数据)
已处理 ANG-Judo (7条数据)
已处理 ANG

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MultipleLocator
from matplotlib.lines import Line2D

# 可视化配置
sns.set_theme(style="darkgrid")
COUNTRIES = ['USA', 'CHN']  # 只保留中美两国

def plot_decomposition_trends(result_df):
    """绘制中美趋势分解图（英文版）"""
    # 配置绘图参数
    plt.rcParams.update({
        'font.sans-serif': ['Arial'],  # 使用英文字体
        'axes.unicode_minus': False,
        'figure.figsize': (12, 6),
        'savefig.dpi': 300
    })

    # 数据预处理
    target_df = result_df[
        (result_df['NOC'].isin(COUNTRIES)) &
        (result_df['Year'] >= 2000)
    ].copy()
    target_df['Year'] = target_df['Year'].astype(int)
    target_df = target_df.sort_values('Year')

    # 创建画布
    fig, ax = plt.subplots()
    
    # 定义可视化元素
    components = ['Total', 'Trend', 'Seasonal']
    line_styles = ['-', '--', '-.']
    colors = sns.color_palette("husl", n_colors=2)
    
    # 绘制趋势线
    for country_idx, country in enumerate(COUNTRIES):
        country_data = target_df[target_df['NOC'] == country]
        
        for comp_idx, comp in enumerate(components):
            ax.plot(
                country_data['Year'],
                country_data[comp],
                label=f'{country} {comp}',
                linestyle=line_styles[comp_idx],
                color=colors[country_idx],
                alpha=0.8,
                linewidth=2 if comp == 'Total' else 1.5
            )

    # 坐标轴设置
    ax.set_xlim(2000, 2020)
    ax.set_ylim(0, target_df['Total'].max()*1.1)
    ax.xaxis.set_major_locator(MultipleLocator(4))
    ax.set_xlabel('Olympic Year', fontsize=12)
    ax.set_ylabel('Medal Count / Components', fontsize=12)
    ax.set_title('China vs USA Olympic Medal Trend Analysis (2000-2020)', 
                pad=15, fontsize=14, fontweight='bold')

    # 优化图例
    handles, labels = ax.get_legend_handles_labels()
    
    # 成分图例（线型）
    comp_legend = plt.legend(
        handles[:3],
        ['Total Medals', 'Long-term Trend', 'Olympic Cycle Effect'],
        title='Components',
        loc='upper left',
        frameon=True,
        shadow=True
    )
    
    # 国家图例（颜色）
    country_legend = plt.legend(
        [Line2D([0], [0], color=colors[i], lw=2) for i in range(2)],
        COUNTRIES,
        title='Countries',
        loc='lower right',
        frameon=True,
        shadow=True
    )
    
    ax.add_artist(comp_legend)
    ax.add_artist(country_legend)

    # 输出图像
    output_path = r"C:\Users\NANA\.jupyter\JupyterProject\MCM_02\china_usa_comparison_en.png"
    plt.savefig(output_path, bbox_inches='tight')
    print(f'Visualization saved to: {output_path}')
    plt.close()

def main():
    # 读取数据并过滤
    result_df = pd.read_csv(r"C:\Users\NANA\.jupyter\JupyterProject\MCM_02\code\data\decomposition_results.csv")
    result_df = result_df[result_df['Year'] >= 2000]
    
    # 执行可视化
    plot_decomposition_trends(result_df)

if __name__ == '__main__':
    main()

Visualization saved to: C:\Users\NANA\.jupyter\JupyterProject\MCM_02\china_usa_comparison_en.png
